# Динамическое исполнение выражений в Python: eval()

Из этого руководства вы узнаете, как в Python работает встроенная функция `eval()`, а также о том, как безопасно и эффективно использовать ее в своих программах.

Данная статья является незначительно сокращенным переводом анлоязычной публикации [Python eval(): Evaluate Expressions Dynamically](https://realpython.com/python-eval-function/). Перевод на русский язык подготовлен для сайта [Библиотеки программиста](https://proglib.io/).

Функция `eval()` полезна, когда вы пытаетесь динамически выполнять выражения Python из какого-либо ввода, представленного в виде строки или объекта скомпилированного кода.

Хотя функция `eval()` – невероятно полезный инструмент, то, что она может выполнять программный код, имеет важные последствия для безопасности, которые следует учитывать перед ее использованием. 

Из этого руководства вы узнаете:
- Как работает `eval()`
- Как использовать `eval()` для динамического выполнения кода
- Как минимизировать риски безопасности, связанные с использованием `eval()`

# Разбираемся в том, как работает eval()

Вы можете использовать встроенyю функцию `eval()` для динамического исполнения выражений из ввода на основе строки или скомпилированного кода. Если вы передаете  в `eval()` строку, то функция анализирует ее, компилирует в [байт-код](https://docs.python.org/3/glossary.html#term-bytecode) и выполняет как выражение Python.

Если вы передаете `eval()` объект скомпилированного кода, то функция сразу переходит к выполнению. Это очень удобно, если вы вызываете `eval()` несколько раз с одним и тем же вводом.

Сигнатура `eval()` определена следующим образом:

```
eval(expression[, globals[, locals]])
```

Первый аргумент `expression` содержит выражение, которое необходимо выполнить. Функция также принимает два необязательных аргумента `globals` и `locals`, о которых мы поговорим в соответствующих разделах. Начнём по порядку с аргумента `expression`.

**Примечание**: Для динамического выполнения кода можно также использовать функцию `exec()`. Основное различие между `eval()` и `exec()` состоит в том, что `eval()` может выполнять лишь выражения, тогда как функции `exec()` можно «скормить» любой фрагмент кода Python.

# Первый аргумент: `expression`

Когда мы вызываем `eval()`, содержание `expression` воспринимается, как выражение Python. Посмотрите на следующие примеры, принимающие строковый ввод:

In [ ]:
eval("2 ** 8")

In [ ]:
eval("1024 + 1024")

In [ ]:
eval("sum([8, 16, 32])")

In [ ]:
x = 100

In [ ]:
eval("x * 2")

При вызове `eval()` со строковым выражением в качестве аргумента, функция возвращает значение, полученное в результате оценки входной строки. По умолчанию `eval()` имеет доступ к глобальным именам, таким как `x` в приведенном выше примере.

Чтобы оценить строковое выражение, `eval()` выполняет следующую последовательность действий:
- Парсинг выражения.
- Компилирование в байт-код.
- Выполнение кода выражения Python.
- Возвращение результата.

Имя аргумента `expression` подчеркивает, что функция работает только с выражениями, но не составными конструкциями. При попытке передачи блока кода вместо выражения будет получено исключение `SyntaxError`:

In [ ]:
x = 100

In [ ]:
eval("if x: print(x)")

Таким образом, в `eval()` нельзя передать конструкции  c `if`, `import`, `def` или `class`, с циклами `for`, `while`. Однако ключевое слово `for` может использоваться в `eval` в случае выражений для генераторов списков.

По той же причине в `eval()` запрещены операции присваивания:

In [ ]:
eval("pi = 3.1416")

`SyntaxError` также возвращается в случаях, когда `eval()` не удается распарсить выражение из-за ошибки в записи:

In [ ]:
eval("5 + 7 *")

В `eval()` также можно передавать объекты кода ([code objects](https://docs.python.org/3/library/stdtypes.html#code-objects)). Чтобы скомпилировать код, который вы собираетесь передать `eval()`, можно использовать [`compile()`](https://docs.python.org/3/library/functions.html#compile). Это встроенная функция, которая может компилировать строку в объект кода или [AST-объект](https://docs.python.org/3/library/ast.html#ast.AST). 

Детали того, как использовать `compile()`, выходят за рамки этого руководства, но здесь мы кратко рассмотрим первые три обязательных аргумента:

1. `source` содержит исходный код, который необходимо скомпилировать. Этот аргумент принимает обычные строки, байтовые строки и объекты AST.
2. `filename` определяет файл, из которого прочитан код. Если используется строчный ввод, значение аргумента должно быть `"<string>"`.
3. `mode` указывает, какой тип объекта кода мы хотим получить. Если нужно обработать код с помощью `eval()`, в качестве значения аргумента указывается `"eval"`.

**Примечание**: для лучшего ознакомления воспользуйтесь [официальной документацией функции `compile()`](https://docs.python.org/3/library/functions.html#compile).

Таким образом, вы можете использовать `compile()` для предоставления объектов кода в `eval()` вместо обычных строк:

In [ ]:
code = compile("5 + 4", "<string>", "eval")
eval(code)

In [ ]:
import math
code = compile("4 / 3 * math.pi * math.pow(25, 3)", "<string>", "eval")
eval(code)

Использование объектов кода полезно при многократном вызове. Если вы предварительно скомпилируете входное выражение, то последующие вызовы `eval()` будут выполняться быстрее, так как не будут повторяться шаги синтаксического анализа и компиляции.

# Второй аргумент: `globals`

Аргумент `globals` опционален. Он содержит словарь, обеспечивающий доступ `eval()` к глобальному пространству имен. С помощью глобальных переменных вы можете указать `eval()`, какие глобальные имена использовать при выполнении выражения.

Глобальные имена – это все те имена, которые доступны в текущей глобальной области или пространстве имен. Вы можете получить к ним доступ из любого места в вашем коде.

Все имена, переданные глобальным переменным в словаре, будут доступны `eval()` во время выполнения.

In [ ]:
x = 100  # Глобальная переменная
eval("x + 100", {"x": x})

In [ ]:
y = 200  # Другая глобальная переменная
eval("x + y", {"x": x})

Любые глобальные имена, определенные вне пользовательского словаря `globals`, не будут доступны изнутри `eval()`, будет вызвано `NameError`.

In [ ]:
eval("x + y", {"x": x, "y": y})

Вы также можете указать имена, которых нет в текущей глобальной области видимости. Чтобы это работало, нужно указать конкретное значение для каждого имени. Тогда `eval()` будет интерпретировать эти имена, как если бы это были глобальные переменные:

In [ ]:
eval("x + y + z", {"x": x, "y": y, "z": 300})

In [ ]:
z  # самой переменной нет в глобальной области видимости

Если вы предоставите `eval()` пользовательский словарь, который не содержит значения для ключа `"__builtins__"`, то ссылка на словарь встроенных функций всё равно будет автоматически добавлена к ключу `"__builtins__"`, прежде чем выражение будет проанализировано. Это гарантирует, что `eval()` имеет полный доступ ко всем встроенным именам Python при оценке выражения.

In [ ]:
eval("sum([2, 2, 2])", {})

In [ ]:
eval("min([1, 2, 3])", {})

In [ ]:
eval("pow(10, 2)", {})

Несмотря на переданный пустой словарь (`{}`), `eval()` имеет доступ к встроенным функциям.

При вызове `eval()` без передачи пользовательского словаря в глобальные переменные аргумент по умолчанию будет использовать словарь, возвращаемый `globals()` в среде, где вызывается `eval()`:

In [ ]:
x = 100  # Глобальная переменная
y = 200  # Другая глобальная переменная
eval("x + y")

Таким образом, передача словаря в аргументе `globals` служит как способ намеренно ограничить область видимость имен для функции `eval()`.

# Третий аргумент: `locals`

Аргумент `locals` также является необязательным аргуменом. В этом случае словарь содержит переменные, которые `eval()` использует в качестве локальных имен при оценке выражения.

Локальными называются те имена (переменные, функции, классы и т.д.), которые вы определяете внутри данной функции. Локальные имена видны только изнутри включающей функции.

In [ ]:
eval("x + 100", {}, {"x": 100})

In [ ]:
eval("x + y", {}, {"x": 100})

Обратите внимание, что для передачи словаря `locals` сначала необходимо предоставить словарь для `globals`. Передача по ключу в случае `eval()` не работает:

In [ ]:
eval("x + 100", locals={"x": 100})

Главное практическое различие между `globals` и `locals` заключается в том, что Python автоматически вставит ключ `"__builtins__"` в `globals`, если этот ключ еще не существует. Cловарь `locals` остается неизменным во время выполнения `eval()`.

# Выполнение выражений с `eval()`

Функция `eval()` используется, когда нужно динамически изменять выражения, а применение других техник и инструментов Python требует избыточных усилий. В этом разделе мы обсудим, как использовать `eval()` для булевых, математических и прочих выражений Python.

## Булевы выражения

Булевы выражения – это выражения Python, которые возвращают логическое значение (`True` или `False`). Обычно они используются в операторе `if` для проверки, является ли какое-либо условие истинным или ложным:

In [ ]:
x = 100
y = 100
eval("x != y")

In [ ]:
eval("x < 200 and y > 100")

In [ ]:
eval("x is y")

In [ ]:
eval("x in {50, 100, 150, 200}")

Зачем же может потребоваться использовать `eval()` вместо непосредственного применения логического выражения? Предположим, нам нужно реализовать условный оператор, но вы хотите на лету менять условие:

In [ ]:
def func(a, b, condition):
    if eval(condition):
        return a + b
    return a - b

In [ ]:
func(2, 4, "a > b")

In [ ]:
func(2, 4, "a < b")

In [ ]:
func(2, 2, "a is b")

Внутри `func()` для оценки предоставленного условия используется функция `eval()`, возвращающая `a + b` или `a - b` в соответствии с результатом оценки.

Теперь представьте, как бы вы реализовали то же поведение без `eval()` для обработки любого логического выражения.

## Математические выражения

Один из распространенных вариантов использования `eval()` в Python – оценка математических выражений из строкового ввода. Например, если вы хотите создать калькулятор на Python, вы можете использовать `eval()`, чтобы оценить вводимые пользователем данные и вернуть результат вычислений:

In [ ]:
eval("5 + 7")

In [ ]:
eval("(5 + 7) / 2")

In [ ]:
import math
eval("math.sqrt(math.pow(10, 2) + math.pow(15, 2))")

## Выражения общего вида

Вы можете использовать `eval()` и с более сложными выражениями Python, включающими вызовы функций, создание объектов, доступ к атрибутам и т. д.

Например, можно вызвать встроенную функцию или функцию, импортированную с помощью стандартного или стороннего модуля. В следующих примерах `eval()` используется для запуска различных системных команд. 

In [ ]:
import subprocess
# Запуск команды echo
eval("subprocess.getoutput('echo Hello, World')")

In [ ]:
# Запуск Firefox (если он установлен)
eval("subprocess.getoutput('firefox')")

Таким образом, можно передавать команды через какой-либо строковый интерфейс (например, форму в браузере) и выполнять код Python.

Однако по этой же причине `eval()` может подвергнуть нас серьезным угрозам безопасности, например, позволит злоумышленнику запускать системные команды или выполнять кода на нашем компьютере. В следующем разделе мы обсудим способы устранения угроз безопасности, связанных с `eval()`.

# Минимизация проблем безопасности, связанных с `eval()`

В связи с проблемами безопасности обычно рекомендуется по возможности не использовать `eval()`. Но если вы решили, что функция необходима, простое практическое правило состоит в том, чтобы никогда не использовать ее для любого ввода, которому нельзя доверять. Сложность в том, чтобы выяснить, каким видам ввода доверять *можно*.

В качестве примера того, как безответственное использование `eval()` может сделать ваш код небезопасным, предположим, что вы хотите создать онлайн-сервис для оценки произвольных выражений Python. Ваш пользователь вводит выражения и нажимает кнопку «Выполнить». Приложение получает пользовательский ввод и передает его для выполнения в `eval()` .

Если вы используете Linux и приложения имеет необходимые разрешения, то злонамеренный пользователь может ввести опасную строку, подобную следующей:

```
"__import__('subprocess').getoutput('rm –rf *')"
```
Выполнение выражения удалит все файлы в текущей директории.

**Примечание**: `__import__()` – это встроенная функция, которая принимает имя модуля в виде строки и возвращает ссылку на объект модуля. `__import__()` - это функция, которая полностью отличается от оператора `import`. Как мы упоминали выше, вы не можете вызвать оператор импорта с помощью `eval()`.

## Ограничение `globals` и `locals`

Вы можете ограничить среду выполнения `eval()`, задавая собственные словари аргументам `globals` и `locals`. Например, пустые словари для обоих аргументов, чтобы `eval()` не мог получить доступ к именам в текущей области или пространстве имен вызывающей стороны:

In [ ]:
x = 100
eval("x * 5", {}, {})

К сожалению, это ограничение не устраняет другие проблемы безопасности, связанные с использованием `eval()`, поскольку остается доступ ко всем встроенным именам и функциям Python.

## Ограничение `__builtins__`

Как мы видели ранее, `eval()` автоматически вставляет ссылку на словарь `__builitins__` в `globals` перед синтаксическим анализом выражения. Злоумышленник может использовать это поведение, используя встроенную функцию `__import __()`, чтобы получить доступ к стандартной библиотеке или любому стороннему модулю, установленному в системе:

In [ ]:
eval("__import__('math').sqrt(25)", {}, {})

In [ ]:
eval("__import__('subprocess').getoutput('echo Hello, World')", {}, {})

Чтобы минимизировать риски, можно переопределить `__builtins__` в `globals`:

In [ ]:
eval("__import__('math').sqrt(25)", {"__builtins__": {}}, {})

## Ограничение имён во входных данных

Однако даже после таких ухищрений Python останется уязвим. Например, можно получить доступ к объекту класса, используя литерал типа, например `""`, `[]`, `{}` или `()`, а также некоторые специальные атрибуты:

In [ ]:
"".__class__.__base__

In [ ]:
[].__class__.__base__

Получив доступ к объекту, можно использовать специальный метод `.__subclasses__()`, чтобы получить доступ ко всем классам, которые наследованы объектом. Вот как это работает:

In [ ]:
for sub_class in ().__class__.__base__.__subclasses__():
    print(sub_class.__name__)

Этот код напечатает большой список классов. Некоторые из этих классов довольно мощные и могут быть чрезвычайно опасны в чужих руках. Это открывает еще одну важную дыру в безопасности, которую вы не сможете закрыть, просто ограничивая окружение `eval()`:

In [ ]:
input_string = """[
    c for c in ().__class__.__base__.__subclasses__()
    if c.__name__ == "range"
    ][0](10)"""

In [ ]:
list(eval(input_string, {"__builtins__": {}}, {}))

Генератор списка в приведенном коде фильтрует классы, которые наследуются от объекта, чтобы вернуть список, содержащий класс `range`. Ему в этом списке соответствует индекс 0. Далее `range` вызывается для создания соответствующего объекта. Это хитрый способ обойти исключение `TypeError`, вызываемое в результате ограничения `"__builtins__"`.

In [ ]:
list(eval(range(10), {"__builtins__": {}}, {}))

Возможное решение этой уязвимости состоит в том, чтобы ограничить использование имен во входных данных  набором безопасных имен либо исключить всякое использование любых имен.

Чтобы реализовать эту технику, необходимо выполнить следующие шаги:
1. Создать словарь, содержащий имена, которые могут использоваться в `eval()`.
2. Скомпилировать входную строку в байт-код, используя `compile()` в режиме `eval`.
3. Проверить `.co_names` в объекте байт-кода, чтобы убедиться, что он содержит только разрешенные имена.
4. Вызвать исключение `NameError`, если пользователь пытается использовать недопустимое имя.

Взглянем на следующую функцию, в которой реализованы все эти шаги:

In [ ]:
def eval_expression(input_string):
    allowed_names = {"sum": sum}
    code = compile(input_string, "<string>", "eval")
    for name in code.co_names:
        if name not in allowed_names:
            raise NameError(f"Использование {name} не разрешено.")
    return eval(code, {"__builtins__": {}}, allowed_names)

Эта функция ограничивает имена, которые можно использовать в `eval()`, именами в словаре `allowed_names`. Для этого функция использует `.co_names` – атрибут объекта кода, содержащий кортеж имен в объекте кода.

Следующие примеры показывают, как написанная нами функция `eval_expression()` работает на практике:

In [ ]:
eval_expression("3 + 4 * 5 + 25 / 2")

In [ ]:
eval_expression("sum([1, 2, 3])")

In [ ]:
eval_expression("pow(10, 2)")

Если нужно полностью запретить использование имен, достаточно переписать `eval_expression()` следующим образом:

In [ ]:
def eval_expression(input_string):
    code = compile(input_string, "<string>", "eval")
    if code.co_names:
        raise NameError(f"Использование имён запрещено.")
    return eval(code, {"__builtins__": {}}, {})

In [ ]:
eval_expression("3 + 4 * 5 + 25 / 2")

In [ ]:
eval_expression("sum([1, 2, 3])")

## Ограничение входных данных до литералов

Типичный пример использования `eval()` в Python – это выполнение выражений, содержащих стандартные литералы Python. Задача настолько распространенная, что стандартная библиотека предоставляет соответствующую функцию `literal_eval()`. Функция не поддерживает операторы, но работает со списками, кортежами, числами, строками и т. д.:

In [ ]:
from ast import literal_eval
literal_eval("15.02")

In [ ]:
literal_eval("[1, 15]")

In [ ]:
literal_eval("{'one': 1, 'two': 2}")

In [ ]:
literal_eval("sum([1, 15]) + 5 + 8 * 2")

# Использование `eval()` совместно с `input()`

В Python 3.x встроенная функция `input()` читает пользовательский ввод из командной строки, преобразует его в строку, удаляет завершающий символ новой строки и возвращает результат вызывающей стороне. Поскольку результатом `input()` является строка, ее можно передать в `eval()` и выполнить как выражение Python:

In [ ]:
eval(input("Введите математическое выражение: "))

Это распространенный вариант использования `eval()`. Он также эмулирует поведеие `input()` в версиях Python 2.x, где функции можно было передать строковое выражение для выполнения (впоследствии от этого отказались из соображений безопасности).

# Построим обработчик математических выражений

Итак, мы узнали, как работает `eval()` в Python и как использовать его на практике. Мы также выяснили, что `eval()` имеет важные последствия для безопасности и что обычно считается хорошей практикой избегать использования `eval()` в коде. Однако в некоторых ситуациях `eval()` может сэкономить много времени и усилий.

В этом разделе вы напишем приложение для оценки математических выражений на лету. Без использования `eval()`, нам потребовалось бы выполнить следующие шаги:

1. Распарсить входное выражение.
2. Преобразовать компоненты выражения в объекты Python (числа, операторы, функции).
3. Объединить всё в исполняемое выражение.
4. Проверить валидность выражения для Python.
5. Выполнить итоговое выражение и вернуть результат вычислений.

Это потребовало бы большой работы, учитывая  разнообразие возможных выражений, которые Python может обрабатывать. К счастью, теперь мы знаем о функции `eval()`.

Всё приложение будет храниться в скрипте `mathrepl.py`. Постепенно мы его заполним необходимым содержимым. Начнем со следующего кода:

In [ ]:
import math

__version__ = "1.0"

ALLOWED_NAMES = {
    k: v for k, v in math.__dict__.items() if not k.startswith("__")
}

PS1 = "mr>>"

WELCOME = f"""
MathREPL {__version__} - обработчик математических выражений на Python!
Введите математическое выражение после приглашения "{PS1}".
Для дополнительной информации используйте команду help.
Чтобы выйти, наберите quit или exit.
"""

USAGE = f"""
Соберите математическое выражение из чисел и операторов.
Можно использовать любые из следующих функций и констант:

{', '.join(ALLOWED_NAMES.keys())}
"""

Модуль `math` мы используем для того, чтобы определить все доступные имена. Три строковые константы используются для вывода строк в интерфейсе программы. Напишем ключевую функцию нашей программы:

In [ ]:
def evaluate(expression):
    """Вычисляет математическое выражение."""
    # Компиляция выражения в байт-код
    code = compile(expression, "<string>", "eval")

    # Валидация доступных имен
    for name in code.co_names:
        if name not in ALLOWED_NAMES:
            raise NameError(f"The use of '{name}' is not allowed")

    return eval(code, {"__builtins__": {}}, ALLOWED_NAMES)

Осталось лишь написать код для взаимодействия с пользователем. В функции  `main()` мы определяем основной цикл программы для чтения введенных данных и расчета математических выражений, введенных пользователем:

In [ ]:
def main():
    """Читает и рассчитывает введенное выражение"""
    print(WELCOME)
    while True:
        # Читаем пользовательский ввод
        try:
            expression = input(f"{PS1} ")
        except (KeyboardInterrupt, EOFError):
            raise SystemExit()

        # Поддержка специальных команд
        if expression.lower() == "help":
            print(USAGE)
            continue
        if expression.lower() in {"quit", "exit"}:
            raise SystemExit()

        # Вычисление выражения и обработка ошибок
        try:
            result = evaluate(expression)
        except SyntaxError:
            # Некорректное выражение
            print("Вы ввели некорректное выражение.")
            continue
        except (NameError, ValueError) as err:
            # Если пользователь попытался использовать неразрешенное имя
            # или неверное значение в переданной функции
            print(err)
            continue

        # Выводим результат, если не было ошибок
        print(f"Результат: {result}")

Проверим результат нашей работы:

In [ ]:
%run -i mathrepl.py

Вот и всё – наш обработчик математических выражений готов! В случае ошибок при вводе или  математически некорректных выражений мы получаем необходимое пояснение. Для самой обработки введенных данных потребовалось лишь несколько строк и функция `eval()`. 

# Заключение
Итак, вы можете использовать `eval()` для выполнения выражений Python из строкового или кодового ввода. Эта встроенная функция полезна, когда вы пытаетесь динамически обновлять выражения Python и хотите избежать проблем с созданием собственного обработчика выражений. Однако пользоваться ей стоит с осторожностью.